In [1]:
import neo4j
import pandas as pd
import geopy.distance

### Search for good osmid pairs

In [2]:
# Node 1 - zakamycze
node_nr_1 = 356926768

In [3]:
# Node 2 - betel
node_nr_2 = 2104495834

## Neo4j Import

In [4]:
NEO4J_URI = "bolt://localhost:7690"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "graph-routing"
NEO4J_DATABASE = "neo4j"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), database=NEO4J_DATABASE)

In [27]:
clear_data_query = "MATCH (n) DETACH DELETE n"

clear_indexes_and_constrains = "CALL apoc.schema.assert({}, {}, true) YIELD label, key RETURN *"

node_constraint_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (i:Intersection) REQUIRE i.osmid IS UNIQUE"

rel_index_query = "CREATE INDEX IF NOT EXISTS FOR ()-[r:ROAD_SEGMENT]-() ON r.osmid"

# TODO: remember ro copy files to import folder in neo4j and change filenames in query

nodes_csv_load = """
    LOAD CSV WITH HEADERS FROM 
    "file:///nodes_neo4j.csv" AS row
    WITH row WHERE row.osmid IS NOT NULL
    MERGE (i:Intersection {osmid: toInteger(row.osmid)})
    SET i.latitude = toFloat(row.y), 
        i.longitude = toFloat(row.x)
    RETURN COUNT(*) as total
    """
    
relationships_csv_load = """
    LOAD CSV WITH HEADERS FROM 
    "file:///relationships_neo4j.csv" AS row
    MATCH (from:Intersection {osmid: toInteger(row.source)})
    MATCH (to:Intersection {osmid: toInteger(row.target)})
    MERGE (from)-[r:ROAD_SEGMENT {osmid: toInteger(row.osmid)}]->(to)
    SET r.length = toFloat(row.length)
    RETURN COUNT(*) AS total
    """


In [28]:
def print_result(results):
    result = [dict(i) for i in results]
    print(result)
    
    result_all = results.consume()
    print(f"result_consumed_after: {result_all.result_consumed_after} ms")
    print(f"result_available_after: {result_all.result_available_after} ms\n")

In [29]:
def delete_database(tx):
    results = tx.run(clear_data_query)
    print_result(results)

def clear_indexes(tx):
    results = tx.run(clear_indexes_and_constrains)
    print_result(results)

In [30]:
def create_constraints(tx):
    results = tx.run(node_constraint_query)
    print_result(results)

    results = tx.run(rel_index_query)
    print_result(results)

In [31]:
def load_nodes(tx):
    results = tx.run(nodes_csv_load)
    print_result(results)

In [32]:
def load_relationshpis(tx):
    results = tx.run(relationships_csv_load)
    print_result(results)

### Clear database

In [33]:
with driver.session() as session:
    session.execute_write(delete_database)
    
with driver.session() as session:
    session.execute_write(clear_indexes)

[]
result_consumed_after: 0 ms
result_available_after: 676 ms

[{'key': 'osmid', 'label': 'ROAD_SEGMENT'}, {'key': 'osmid', 'label': 'Intersection'}]
result_consumed_after: 4 ms
result_available_after: 38 ms



### Create constraints

In [34]:
with driver.session() as session:
    session.execute_write(create_constraints)

[]
result_consumed_after: 0 ms
result_available_after: 22 ms

[]
result_consumed_after: 0 ms
result_available_after: 11 ms



### Load data

In [35]:
with driver.session() as session:
    session.execute_write(load_nodes)

[{'total': 333120}]
result_consumed_after: 5382 ms
result_available_after: 142 ms



In [36]:
with driver.session() as session:
    session.execute_write(load_relationshpis)

[{'total': 721628}]
result_consumed_after: 21499 ms
result_available_after: 187 ms



### Search queries

```
EXPLAIN PROFILE
```

In [37]:
def dijkstra_query(tx, osmid_1, osmid_2):
    dijkstra_query = f"""
        MATCH (source:Intersection {{osmid: {osmid_1}}}) 
        MATCH (target:Intersection {{osmid: {osmid_2}}})
        CALL apoc.algo.dijkstra(source, target, "ROAD_SEGMENT", "length")
        YIELD path, weight
        RETURN path, weight
        """
    results = tx.run(dijkstra_query)
    print_result(results)

In [38]:
def astar_query(tx, osmid_1, osmid_2):
    astar_query = f"""
        MATCH (source:Intersection {{osmid: {osmid_1}}}) 
        MATCH (target:Intersection {{osmid: {osmid_2}}})
        CALL apoc.algo.aStar(source, target, "ROAD_SEGMENT", "length", "latitude", "longitude")
        YIELD path, weight
        RETURN path, weight
        """
    results = tx.run(astar_query)
    print_result(results)

In [35]:
gdf_nodes = pd.read_csv("./data/krakow_nodes_neo4j.csv")

In [216]:
# two random osmid
start_and_finish = gdf_nodes.sample(2)

osmid_1 = start_and_finish.iloc[0]["osmid"]
osmid_2 = start_and_finish.iloc[1]["osmid"]

coords_1 = start_and_finish.iloc[0]["y"], start_and_finish.iloc[0]["x"]
coords_2 = start_and_finish.iloc[1]["y"], start_and_finish.iloc[1]["x"]

print(f"osmid_1: {osmid_1}, osmid_2: {osmid_2}, distance: {geopy.distance.distance(coords_1, coords_2).km} km")

osmid_1: 10298662254.0, osmid_2: 2139306598.0, distance: 24.768543728456766 km


In [39]:
# node_nr_1 = 356926768
# node_nr_2 = 2104495834

node_nr_1 = 356926768
node_nr_2 = 2104495834

In [42]:
with driver.session() as session:
    session.execute_write(dijkstra_query, node_nr_1, node_nr_2)

[{'path': <Path start=<Node element_id='4:72af26a5-326b-4f9b-ad24-e33bf1c74118:352147' labels=frozenset({'Intersection'}) properties={'osmid': 356926768, 'latitude': 50.0607108, 'longitude': 19.8386575}> end=<Node element_id='4:72af26a5-326b-4f9b-ad24-e33bf1c74118:411311' labels=frozenset({'Intersection'}) properties={'osmid': 2104495834, 'latitude': 50.0636309, 'longitude': 19.9348424}> size=297>, 'weight': 7793.594000000004}]
result_consumed_after: 546 ms
result_available_after: 2 ms



In [43]:
with driver.session() as session:
    session.execute_write(astar_query, node_nr_1, node_nr_2)

[{'path': <Path start=<Node element_id='4:72af26a5-326b-4f9b-ad24-e33bf1c74118:352147' labels=frozenset({'Intersection'}) properties={'osmid': 356926768, 'latitude': 50.0607108, 'longitude': 19.8386575}> end=<Node element_id='4:72af26a5-326b-4f9b-ad24-e33bf1c74118:411311' labels=frozenset({'Intersection'}) properties={'osmid': 2104495834, 'latitude': 50.0636309, 'longitude': 19.9348424}> size=297>, 'weight': 7793.594000000004}]
result_consumed_after: 56 ms
result_available_after: 2 ms



### Add addresses - TODO

In [ ]:
# We'll use apoc.load.json to import a JSON file of address data

address_constraint_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Address) REQUIRE a.id IS UNIQUE"

add_addresses_query = """
CALL apoc.periodic.iterate(
  'CALL apoc.load.json("/home/pcend/Piotr/kod/lisboa-routing/data/lisboa.geojson") YIELD value',
  'MERGE (a:Address {id: value.properties.id})
SET a.location = 
  point(
      {latitude: value.geometry.coordinates[1], longitude: value.geometry.coordinates[0]}),
    a.full_address = value.properties.number + " " + value.properties.street + " " + value.properties.city + ", CA " + value.properties.postcode

SET a += value.properties',
  {batchSize:10000, parallel:true})
"""

# Next, connect each address to the road network at the nearest intersection

near_intersection_query = """
CALL apoc.periodic.iterate(
  'MATCH (p:Address) WHERE NOT EXISTS ((p)-[:NEAREST_INTERSECTION]->(:Intersection)) RETURN p',
  'CALL {
  WITH p
  MATCH (i:Intersection)
  USING INDEX i:Intersection(location)
  WHERE point.distance(i.location, p.location) < 200

  WITH i
  ORDER BY point.distance(p.location, i.location) ASC 
  LIMIT 1
  RETURN i
}
WITH p, i

MERGE (p)-[r:NEAREST_INTERSECTION]->(i)
SET r.length = point.distance(p.location, i.location)
RETURN COUNT(p)',
  {batchSize:1000, parallel:false})
"""

# Create a full text index to enable search in our web app

full_text_query = "CREATE FULLTEXT INDEX search_index IF NOT EXISTS FOR (p:PointOfInterest|Address) ON EACH [p.name, p.full_address]"

In [ ]:
def enrich_addresses(tx):
    results = tx.run(add_addresses_query)        
    results = tx.run(near_intersection_query)


In [ ]:
# with driver.session() as session:
#     session.execute_write(enrich_addresses)

In [ ]:
# with driver.session() as session:
#     results = session.execute_write(lambda tx: tx.run(full_text_query))


### Query with address

```
MATCH (a:Address)-[:NEAREST_INTERSECTION]->(source:Intersection)
WHERE a.full_address CONTAINS "410 E 5TH AVE SAN MATEO, CA"
MATCH 
  (poi:Address)-[:NEAREST_INTERSECTION]->(dest:Intersection) 
WHERE poi.full_address CONTAINS "111 5TH AVE"
CALL apoc.algo.dijkstra(source, dest, "ROAD_SEGMENT", "length") 
YIELD weight, path
RETURN **
```


```
CALL db.index.fulltext.queryNodes("search_index", $searchString) 
YIELD node, score
RETURN coalesce(node.name, node.full_address) AS value, score, labels(node)[0] AS label, node.id AS id
ORDER BY score DESC LIMIT 25
```

```
MATCH (to {id: $dest})-[:NEAREST_INTERSECTION]->(source:Intersection) 
MATCH (from {id: $source})-[:NEAREST_INTERSECTION]->(target:Intersection)
CALL apoc.algo.dijkstra(source, target, 'ROAD_SEGMENT', 'length')
YIELD path, weight
RETURN [n in nodes(path) | [n.location.latitude, n.location.longitude]] AS route
```
